In [1]:
!pip install xmltodict

In [2]:
import pandas as pd
import random
import json
import numpy as np
import sys
import requests
import xmltodict
from PIL import Image
import matplotlib.pyplot as plt
import torch.optim as optim
import shutil
import copy
import time
from torch.utils.data import Dataset
import xml.etree.ElementTree as et
from torchvision import transforms
from torch.utils.data import DataLoader

import torch
import torch.nn as nn

from torchvision.models import * 
from torch.optim import lr_scheduler

import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import gc
gc.collect()
torch.cuda.empty_cache()


c:\Users\white\AppData\Local\Programs\Python\Python37\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


#### <b>Mix Style Library</b>

In [4]:
import random
from contextlib import contextmanager
import torch
import torch.nn as nn


def deactivate_mixstyle(m):
    if type(m) == MixStyle:
        m.set_activation_status(False)


def activate_mixstyle(m):
    if type(m) == MixStyle:
        m.set_activation_status(True)


def random_mixstyle(m):
    if type(m) == MixStyle:
        m.update_mix_method("random")


def crossdomain_mixstyle(m):
    if type(m) == MixStyle:
        m.update_mix_method("crossdomain")


@contextmanager
def run_without_mixstyle(model):
    # Assume MixStyle was initially activated
    try:
        model.apply(deactivate_mixstyle)
        yield
    finally:
        model.apply(activate_mixstyle)


@contextmanager
def run_with_mixstyle(model, mix=None):
    # Assume MixStyle was initially deactivated
    if mix == "random":
        model.apply(random_mixstyle)

    elif mix == "crossdomain":
        model.apply(crossdomain_mixstyle)

    try:
        model.apply(activate_mixstyle)
        yield
    finally:
        model.apply(deactivate_mixstyle)


class MixStyle(nn.Module):
    """MixStyle.
    Reference:
      Zhou et al. Domain Generalization with MixStyle. ICLR 2021.
    """

    def __init__(self, p=0.5, alpha=0.1, eps=1e-6, mix="random"):
        """
        Args:
          p (float): probability of using MixStyle.
          alpha (float): parameter of the Beta distribution.
          eps (float): scaling parameter to avoid numerical issues.
          mix (str): how to mix.
        """
        super().__init__()
        self.p = p
        self.beta = torch.distributions.Beta(alpha, alpha)
        self.eps = eps
        self.alpha = alpha
        self.mix = mix
        self._activated = True

    def __repr__(self):
        return (
            f"MixStyle(p={self.p}, alpha={self.alpha}, eps={self.eps}, mix={self.mix})"
        )

    def set_activation_status(self, status=True):
        self._activated = status

    def update_mix_method(self, mix="random"):
        self.mix = mix

    def forward(self, x):
        if not self.training or not self._activated:
            return x

        if random.random() > self.p:
            return x

        B = x.size(0)

        mu = x.mean(dim=[2, 3], keepdim=True)
        var = x.var(dim=[2, 3], keepdim=True)
        sig = (var + self.eps).sqrt()
        mu, sig = mu.detach(), sig.detach()
        x_normed = (x-mu) / sig

        lmda = self.beta.sample((B, 1, 1, 1))
        lmda = lmda.to(x.device)

        if self.mix == "random":
            # random shuffle
            perm = torch.randperm(B)

        elif self.mix == "crossdomain":
            # split into two halves and swap the order
            perm = torch.arange(B - 1, -1, -1)  # inverse index
            perm_b, perm_a = perm.chunk(2)
            perm_b = perm_b[torch.randperm(perm_b.shape[0])]
            perm_a = perm_a[torch.randperm(perm_a.shape[0])]
            perm = torch.cat([perm_b, perm_a], 0)

        else:
            raise NotImplementedError

        mu2, sig2 = mu[perm], sig[perm]
        mu_mix = mu*lmda + mu2 * (1-lmda)
        sig_mix = sig*lmda + sig2 * (1-lmda)

        return x_normed*sig_mix + mu_mix

#### <b>Parameter</b>

In [5]:
Parameter = {
      "model":"resnet18",
      "weight":"IMAGENET1K_V1",
      "Loss Function":"CrossEntropyLoss",
      "optimizer":{"model":"Adam","lr":0.001,"momentum":0.9},
      "scheduler":{"model":"StepLR","Period":7,"gamma":0.1},
      "data_condition":{"mode":0}, # # 0 : y축 분류, 1 : x축 분류, 2 : x,y 축 분류인데 test 를 train이 아닌 전부로, 3 : x,y 축 분류인데 test 를 ytest 에 해당하는 값들로
      "data_feature":{"xtrain_angle":[i for i in range(1,15)],"xtest_angle":[i for i in range(15,25)],"ytrain_angle":[0,30],"ytest_angle":[60]},
      "Domain_Generalization":0, # Domain Generalization (도메인 일반화, 데이터 증진 or mixstyle 등)
    }

def set_parameter(Hp):
    global Parameter
    Parameter=Hp
    return Hp

#### <b>ResNet Architecture</b>

In [6]:
from functools import partial
from typing import Any, Callable, List, Optional, Type, Union

import torch
import torch.nn as nn
from torch import Tensor

from torchvision.transforms._presets import ImageClassification
from torchvision.utils import _log_api_usage_once
from torchvision.models._api import register_model, Weights, WeightsEnum
from torchvision.models._meta import _IMAGENET_CATEGORIES
from torchvision.models._utils import _ovewrite_named_param, handle_legacy_interface

In [7]:
__all__ = [
    "ResNet",
    "ResNet18_Weights",
    "ResNet34_Weights",
    "resnet18",
    "resnet34",
]


def conv3x3(in_planes: int, out_planes: int, stride: int = 1, groups: int = 1, dilation: int = 1) -> nn.Conv2d:
    """3x3 convolution with padding"""
    return nn.Conv2d(
        in_planes,
        out_planes,
        kernel_size=3,
        stride=stride,
        padding=dilation,
        groups=groups,
        bias=False,
        dilation=dilation,
    )


def conv1x1(in_planes: int, out_planes: int, stride: int = 1) -> nn.Conv2d:
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


class BasicBlock(nn.Module):
    expansion: int = 1

    def __init__(
        self,
        inplanes: int,
        planes: int,
        stride: int = 1,
        downsample: Optional[nn.Module] = None,
        groups: int = 1,
        base_width: int = 64,
        dilation: int = 1,
        norm_layer: Optional[Callable[..., nn.Module]] = None,
    ) -> None:
        super().__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if groups != 1 or base_width != 64:
            raise ValueError("BasicBlock only supports groups=1 and base_width=64")
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in BasicBlock")
        # Both self.conv1 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x: Tensor) -> Tensor:
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    # Bottleneck in torchvision places the stride for downsampling at 3x3 convolution(self.conv2)
    # while original implementation places the stride at the first 1x1 convolution(self.conv1)
    # according to "Deep residual learning for image recognition"https://arxiv.org/abs/1512.03385.
    # This variant is also known as ResNet V1.5 and improves accuracy according to
    # https://ngc.nvidia.com/catalog/model-scripts/nvidia:resnet_50_v1_5_for_pytorch.

    expansion: int = 4

    def __init__(
        self,
        inplanes: int,
        planes: int,
        stride: int = 1,
        downsample: Optional[nn.Module] = None,
        groups: int = 1,
        base_width: int = 64,
        dilation: int = 1,
        norm_layer: Optional[Callable[..., nn.Module]] = None,
    ) -> None:
        super().__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        width = int(planes * (base_width / 64.0)) * groups
        # Both self.conv2 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv1x1(inplanes, width)
        self.bn1 = norm_layer(width)
        self.conv2 = conv3x3(width, width, stride, groups, dilation)
        self.bn2 = norm_layer(width)
        self.conv3 = conv1x1(width, planes * self.expansion)
        self.bn3 = norm_layer(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x: Tensor) -> Tensor:
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class ResNet(nn.Module):
    def __init__(
        self,
        block: Type[Union[BasicBlock, Bottleneck]],
        layers: List[int],
        num_classes: int = 1000,
        zero_init_residual: bool = False,
        groups: int = 1,
        width_per_group: int = 64,
        replace_stride_with_dilation: Optional[List[bool]] = None,
        norm_layer: Optional[Callable[..., nn.Module]] = None,
    ) -> None:
        super().__init__()
        _log_api_usage_once(self)
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer

        self.inplanes = 64
        self.dilation = 1
        if replace_stride_with_dilation is None:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError(
                "replace_stride_with_dilation should be None "
                f"or a 3-element tuple, got {replace_stride_with_dilation}"
            )

            
        if Parameter["Domain_Generalization"]==1:
            #print("SDFSDF")
            ## MixStyle 정의
            self.mixstyle = MixStyle(p=0.5, alpha=0.1)


        self.groups = groups
        self.base_width = width_per_group
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2, dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2, dilate=replace_stride_with_dilation[1])
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2, dilate=replace_stride_with_dilation[2])
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="relu")
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck) and m.bn3.weight is not None:
                    nn.init.constant_(m.bn3.weight, 0)  # type: ignore[arg-type]
                elif isinstance(m, BasicBlock) and m.bn2.weight is not None:
                    nn.init.constant_(m.bn2.weight, 0)  # type: ignore[arg-type]

    def _make_layer(
        self,
        block: Type[Union[BasicBlock, Bottleneck]],
        planes: int,
        blocks: int,
        stride: int = 1,
        dilate: bool = False,
    ) -> nn.Sequential:
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(
            block(
                self.inplanes, planes, stride, downsample, self.groups, self.base_width, previous_dilation, norm_layer
            )
        )
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(
                block(
                    self.inplanes,
                    planes,
                    groups=self.groups,
                    base_width=self.base_width,
                    dilation=self.dilation,
                    norm_layer=norm_layer,
                )
            )

        return nn.Sequential(*layers)

    def _forward_impl(self, x: Tensor) -> Tensor:
        # See note [TorchScript super()]
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        ## update 
        if Parameter["Domain_Generalization"]==1:
            #print("#####")
            x = self.mixstyle(x)
            
        x = self.layer2(x)

        if Parameter["Domain_Generalization"]==1:
            #print("@@@@@")
            x = self.mixstyle(x)

        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

    def forward(self, x: Tensor) -> Tensor:
        return self._forward_impl(x)


def _resnet(
    block: Type[Union[BasicBlock, Bottleneck]],
    layers: List[int],
    weights: Optional[WeightsEnum],
    progress: bool,
    **kwargs: Any,
) -> ResNet:
    if weights is not None:
        #print(weights)
        _ovewrite_named_param(kwargs, "num_classes", len(weights.meta["categories"]))

    model = ResNet(block, layers, **kwargs)

    if weights is not None:
        model.load_state_dict(weights.get_state_dict(progress=progress))

    return model


_COMMON_META = {
    "min_size": (1, 1),
    "categories": _IMAGENET_CATEGORIES,
}


class ResNet18_Weights(WeightsEnum):
    IMAGENET1K_V1 = Weights(
        url="https://download.pytorch.org/models/resnet18-f37072fd.pth",
        transforms=partial(ImageClassification, crop_size=224),
        meta={
            **_COMMON_META,
            "num_params": 11689512,
            "recipe": "https://github.com/pytorch/vision/tree/main/references/classification#resnet",
            "_metrics": {
                "ImageNet-1K": {
                    "acc@1": 69.758,
                    "acc@5": 89.078,
                }
            },
            "_docs": """These weights reproduce closely the results of the paper using a simple training recipe.""",
        },
    )
    DEFAULT = IMAGENET1K_V1


class ResNet34_Weights(WeightsEnum):
    IMAGENET1K_V1 = Weights(
        url="https://download.pytorch.org/models/resnet34-b627a593.pth",
        transforms=partial(ImageClassification, crop_size=224),
        meta={
            **_COMMON_META,
            "num_params": 21797672,
            "recipe": "https://github.com/pytorch/vision/tree/main/references/classification#resnet",
            "_metrics": {
                "ImageNet-1K": {
                    "acc@1": 73.314,
                    "acc@5": 91.420,
                }
            },
            "_docs": """These weights reproduce closely the results of the paper using a simple training recipe.""",
        },
    )
    DEFAULT = IMAGENET1K_V1


@handle_legacy_interface(weights=("pretrained", ResNet18_Weights.IMAGENET1K_V1))
def Resnet18(*, weights: Optional[ResNet18_Weights] = None, progress: bool = True, **kwargs: Any) -> ResNet:
    
    weights = ResNet18_Weights.verify(weights)
    return _resnet(BasicBlock, [2, 2, 2, 2], weights, progress, **kwargs)


@handle_legacy_interface(weights=("pretrained", ResNet34_Weights.IMAGENET1K_V1))
def Resnet34(*, weights: Optional[ResNet34_Weights] = None, progress: bool = True, **kwargs: Any) -> ResNet:
    
    weights = ResNet34_Weights.verify(weights)
    return _resnet(BasicBlock, [3, 4, 6, 3], weights, progress, **kwargs)

In [8]:
# The dictionary below is internal implementation detail and will be removed in v0.15
from torchvision.models._utils import _ModelURLs


model_urls = _ModelURLs(
    {
        "resnet18": ResNet18_Weights.IMAGENET1K_V1.url,
        "resnet34": ResNet34_Weights.IMAGENET1K_V1.url
    }
)

In [9]:
#resnet18()

In [10]:
#from google.colab import drive
#drive.mount('/content/drive')

특정 폴더 기준으로 들어감, inital_path 설정해야됨

In [11]:
class market_product_identification():

    def __init__(self,inital_path,using_category,data_feature,Data_augmentation):

        # Data Frame 없이 할 수 있음 (바꾸기)
        # 각도별 path를 저장하고 불러올때 불러오기
        # Data to use in xml file
        self.category=using_category

        self.Data_augmentation=Data_augmentation

        self.a0_path=[]
        self.a30_path=[]
        self.a60_path=[]
        self.nb={}
        self.nl_idx={}

        #file initial position
        self.inital_path=inital_path

        self.data_path=self.inital_path+"dataset/"

        #DataFrame creation and data labeling
        self.object_type,self.object_stack,self.object_count,self.DataFrame=self.new_label(self.produce_DataFrame())
        #print(self.DataFrame)
        #print(self.DataFrame)
        #print(self.DataFrame.shape)




        self.ytrain_angle=data_feature["ytrain_angle"]
        self.ytest_angle=data_feature["ytest_angle"]

        #print(self.DataFrame)
        self.train,self.test=self.data_dividing(self.DataFrame)

        #print(f"train shape : {self.train.shape}")
        #print(f"test shape : {self.test.shape}")

        #"""
        #Generate transform for each data set
        self.train_transform,self.val_transform,self.test_transform=self.transform(256)




    # return market_product_identification's variable


    # Find the file corresponding to the path
    def find_list(self,path):
        return os.listdir(path)
    
    # DataFrame creation
    def produce_DataFrame(self):

      #print(self.nb)
    

      img_path=self.data_path


      #label_path=self.inital_path+"sample_data_food/라벨링데이터/"
      #image_path=self.inital_path+"sample_data_food/원천데이터/"

      #print("produce_dataframe")

      dataframe=np.array([])
      dataframe = np.empty((0,len(self.category)), int)


      

      for name in self.find_list(img_path):

          for angle in ['0','30','60']:

            for data in self.find_list(img_path+name+'/'+angle+'/'):

              DT=data.split("_")
            
              #'prod_nm','file_name','yangle','path','rotation'
              Da=[name, data, int(DT[1]), img_path+name+'/'+angle+'/'+data,0]
              dataframe=np.append(dataframe,np.array([Da]),axis=0)
              self.nb[name]=str(data.split("_")[0])
              self.nl_idx[str(data.split("_")[0])]=name

              if angle=='0':
                self.a0_path.append(img_path+name+'/'+angle+'/'+data)
              elif angle=='30':
                self.a30_path.append(img_path+name+'/'+angle+'/'+data)
              elif angle=='60':
                self.a60_path.append(img_path+name+'/'+angle+'/'+data)


            
      return pd.DataFrame(data=dataframe,columns=self.category)
    
    # data labeling
    def new_label(self,DataFrame):
        object_type={}
        idx=0
        object_stack=[]
        for i in range(DataFrame.shape[0]):
            data=DataFrame.iloc[i]["prod_nm"]
            ZZ=DataFrame.iloc[i]["file_name"]
            
            if not data in object_type:
                object_type[data]=idx
                object_stack.append({'0':0,'30':0,'60':0})
                object_stack[object_type[data]][ZZ.split("_")[1]]+=1
                idx+=1
            else:
                #print(ZZ.split("_"))
                object_stack[object_type[data]][ZZ.split("_")[1]]+=1
            #print(object_stack)

            A=DataFrame.iloc[i]["prod_nm"]
            self.nl_idx[self.nb[A]]=object_type[data]
            DataFrame.iloc[i]["prod_nm"]=object_type[data]
 
        
        DataFrame= DataFrame.sample(frac=1)  # row 전체 shuffle
        DataFrame = DataFrame.sample(frac=1).reset_index(drop=True)  # shuffling하고 index reset


        #print("finish")
        #print(object_stack)

        return object_type,object_stack,len(object_stack),DataFrame
    

    # Dividing of DataFrame
    def data_dividing(self,DataFrame):
        #self.object_type,self.object_stack,self.obejct_count,self.DataFrame

        train=np.array([])
        train = np.empty((0,len(self.category)), int)
        test=np.array([])
        test = np.empty((0,len(self.category)), int)
        #print(dict(pd.iloc[0]))
        for i in range(DataFrame.shape[0]):
            data=DataFrame.iloc[i]["file_name"]

            Ip=[]
            for g in range(len(self.category)):Ip.append(DataFrame.iloc[i][self.category[g]])
            ya=int(DataFrame.iloc[i]["yangle"])
            #print(i,xa,ya)

            if ya in self.ytrain_angle:
                train=np.append(train,np.array([Ip]),axis=0)

            elif ya in self.ytest_angle:
                test=np.append(test,np.array([Ip]),axis=0)
  





        train=pd.DataFrame(data=train,columns=self.category)
        test=pd.DataFrame(data=test,columns=self.category)
        #print("train")
        #print(train)
        #print("Test")
        #print(test)

        return train,test
    
    #Generate transform for each data set
    def transform(self,size=256):
        train_transform = transforms.Compose([
            transforms.Resize(size),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]) 
        ])

        val_transform = transforms.Compose([
            transforms.Resize(size),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]) 
        ])

        test_transform = transforms.Compose([
            transforms.Resize(size),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]) 
        ])
        return train_transform,val_transform,test_transform
    
  


In [12]:
class CustomDataset(Dataset):
  def __init__(self,label,transform=None,tt={"mode":"train","angle":-1},a0_path=[],a30_path=[],a60_path=[],nl_idx=0,Data_augmentation={}):
    self.label_data=label
    self.nl_idx=nl_idx
    self.Data_augmentation=Data_augmentation
    self.transform=transform
    self.path={"0":a0_path,"30":a30_path,"60":a60_path}
    self.path_l={"0":len(a0_path),"30":len(a30_path),"60":len(a60_path)}
    self.mode=tt["mode"]
    self.test_angle=[]
    self.train_angle=[]
    for angle in ["0","30","60"]:
      if angle==str(tt['angle']):
        self.test_angle.append(angle)
      else:
        self.train_angle.append(angle)

    self.da_cnt=0
    J=list(self.Data_augmentation)
    for i in range(len(J)):
      self.da_cnt+=self.Data_augmentation[J[i]]

    #print("#@#@#@#@#@#@#@#@#")
    #print(self.path_l)
    #print(self.train_angle)
    #print(self.test_angle)

    #self.test_angle=test_angle



  def __len__(self):
    pr=0
    #return len(self.label_data)
    if self.mode=="train":
      pr = self.path_l[self.train_angle[0]]+self.path_l[self.train_angle[1]]
    else:
      pr = self.path_l[self.test_angle[0]]
    #return self.path_l["0"]+self.path_l["30"]+self.path_l["60"]#len(self.label_data)

    pr*=(self.da_cnt+1)
    return pr

  def __getitem__(self,idx):
    #label=int(self.label_data.iloc[idx]["prod_nm"])
    #img_path=self.label_data.iloc[idx]["path"]
    K=idx//(self.da_cnt+1)


    #"""
    if self.mode=="train":
      if K>=self.path_l[self.train_angle[0]]:
        img_path=self.path[self.train_angle[1]][(K-self.path_l[self.train_angle[0]])]
      else:
        img_path=self.path[self.train_angle[0]][K]

    elif self.mode=="test":
      img_path=self.path[self.test_angle[0]][K]
    #"""

    AA=img_path.split("/")
    #print(AA[len(AA)-1].split("_")[0])

    if (self.mode=="train" and str(AA[len(AA)-1].split("_")[1]) in self.test_angle) or (self.mode=="test" and str(AA[len(AA)-1].split("_")[1]) in self.train_angle):
      print("error")

    label=self.nl_idx[AA[len(AA)-1].split("_")[0]]

    #if self.lable_data.iloc[idx]["yangle"]==self.test_angle:
    #print("###",img_path,label)

    img=Image.open(img_path)


    #"Data_augmentation":{"image_rotation":0,"ColorJitter":0,}
    J=list(self.Data_augmentation)
    if self.Data_augmentation["image_rotation"]==1 and idx%(self.da_cnt+1)==1:
      #print("#")
      rotater=transforms.RandomRotation(degrees=(0,180))
      IMG=rotater(img)
      img=IMG
      
    elif self.Data_augmentation["ColorJitter"]==1 and idx%(self.da_cnt+1)==2:
      #print("##")
      jitter = transforms.ColorJitter(brightness=.5, hue=.3)
      IMG=jitter(img)
      img=IMG

    elif self.Data_augmentation["RandomHorizontalFlip"]==1 and idx%(self.da_cnt+1)==3:
      #print("###")
      hflipper = transforms.RandomHorizontalFlip(p=0.5)
      IMG=hflipper(img)

    elif self.Data_augmentation["RandomVerticalFlip"]==1 and idx%(self.da_cnt+1)==4:
      vflipper = transforms.RandomVerticalFlip(p=0.5)
      IMG=vflipper(img)
      img=IMG

    elif self.Data_augmentation["RandomGrayscale"]==1 and idx%(self.da_cnt+1)==5:
      scaler = transforms.RandomGrayscale(p=0.75)
      IMG=scaler(img)
      img=IMG
    


    if self.transform:
      img=self.transform(img)


    return img,label





In [13]:
def produce_Dataset(train_v,test_v,batch_size=64,test_angle=0, a0_path=[], a30_path=[], a60_path=[],nl_idx=0,Data_augmentation={}):

  train_dataset = CustomDataset(train_v["label_data"],train_v["transform"],tt={"mode":"train","angle":test_angle},a0_path=a0_path,a30_path=a30_path,a60_path=a60_path,nl_idx=nl_idx,Data_augmentation=Data_augmentation)
  train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)

  test_dataset = CustomDataset(test_v["label_data"],test_v["transform"],tt={"mode":"test","angle":test_angle},a0_path=a0_path,a30_path=a30_path,a60_path=a60_path,nl_idx=nl_idx,Data_augmentation=Data_augmentation)
  test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
  
  return {"dataset":train_dataset,"dataloader":train_dataloader},{"dataset":test_dataset,"dataloader":test_dataloader}


In [14]:
def train_model(model, criterion, optimizer, scheduler, learning_dataloader,learning_dataset_size,batch_size,train_size,Data_augmentation):

  model.train()  # 모델을 학습 모드로 설정


  running_loss = 0.0
  running_corrects = 0.0 #[0]*(len(MPI.object_stack)+2)
  Cor=[0]*(len(train_size)+2)


  # 데이터를 반복
  for i, batch in enumerate(learning_dataloader):
    #print(f"i : {i}, batch : {batch}.")
      
    inputs,labels=batch

    inputs = inputs.to(device)

    labels = labels.type(torch.LongTensor).to(device)


    outputs = model(inputs)

    _, preds = torch.max(outputs, 1)

    loss = criterion(outputs, labels)

    optimizer.zero_grad()

    #loss.requires_grad_(True)
    loss.backward()
    #"""
      
    optimizer.step()
    
    # 통계
    running_loss += loss.item() * inputs.size(0)
    running_corrects += torch.sum(preds == labels.data)

    #print(preds[0].item())
    #print(labels.data[0].item())
    #print(len(labels.data))
    #print("##train##")
    #print(len(preds),len(labels.data))
    for i in range(len(labels.data)):
      if int(preds[i].item())==int(labels.data[i].item()):
        Cor[int(labels.data[i].item())]+=1


  scheduler.step()

  epoch_acc=0
  #print("train")
  #print(Cor)
  #print(train_size)
  #print(rotation)


  da_cnt=0
  J=list(Data_augmentation)
  for i in range(len(J)):
    da_cnt+=Data_augmentation[J[i]]


  for i in range(len(train_size)):
    epoch_acc+=Cor[i]/(train_size[i]*(da_cnt+1))


  
  #print("train new")
  #print(Cor,train_size)

  
  

  epoch_loss = running_loss / learning_dataset_size
  #print("train",running_corrects.double(),learning_dataset_size)
  #epoch_acc = running_corrects.double() / learning_dataset_size
  epoch_acc=epoch_acc / len(train_size)
  
  print(f'train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

  return epoch_loss,epoch_acc,model,optimizer,scheduler

In [15]:
def test_model(model, criterion,learning_dataloader,learning_dataset_size,test_size,Data_augmentation):
  model.eval()  # 모델을 학습 모드로 설정

  running_loss = 0.0
  running_corrects = 0

  Cor=[0]*(len(test_size)+2)

  # 데이터를 반복
  for i, batch in enumerate(learning_dataloader):
      
    inputs,labels=batch
    inputs = inputs.to(device)
    labels = labels.type(torch.LongTensor).to(device)


    outputs = model(inputs)
    _, preds = torch.max(outputs, 1)
    loss = criterion(outputs, labels)
    
    # 통계
    running_loss += loss.item() * inputs.size(0)
    running_corrects += torch.sum(preds == labels.data)

    #print(preds[0].item())
    #print(labels.data[0].item())
    #print(len(labels.data))
    #print("##test##")
    #print(len(preds),len(labels.data))
    for i in range(len(labels.data)):
      if int(preds[i].item())==int(labels.data[i].item()):
        Cor[int(labels.data[i].item())]+=1

  epoch_acc=0

  #print("test new")
  #print(Cor,test_size)

  da_cnt=0
  J=list(Data_augmentation)
  for i in range(len(J)):
    da_cnt+=Data_augmentation[J[i]]


  for i in range(len(test_size)):
    epoch_acc+=Cor[i]/(test_size[i]*(da_cnt+1))

  #print("test")
  #print(Cor)
  #print(test_size)
  #print(rotation)

  epoch_loss = running_loss / learning_dataset_size
  #print("test",running_corrects.double(),learning_dataset_size)
  #epoch_acc = running_corrects.double() / learning_dataset_size 
  epoch_acc=epoch_acc / len(test_size)

  print(f'test Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

  return epoch_loss,epoch_acc,model

In [16]:
class dl_Model():
  def __init__(self,class_names,device):
    self.class_names=class_names
    self.device=device

  def produce_model(self,Hyperparameter):
    
    set_parameter(Hyperparameter)

    if Hyperparameter["model"]=="resnet18":
      return self.RESNET18(Hyperparameter)
    elif Hyperparameter["model"]=="resnet34":
      return self.RESNET34(Hyperparameter)
    elif Hyperparameter["model"]=="alexnet":
      return self.ALEXNET(Hyperparameter)
    elif Hyperparameter["model"]=="densenet121":
      return self.DENSENET121(Hyperparameter)
    
  def DENSENET121(self,Hyperparameter):
    
    """
    {
      "model":"densenet121",
      "weight":"IMAGENET1K_V1",
      "Loss Function":"CrossEntropyLoss",
      "optimizer":{"model":"SGD","lr":0.001,"momentum":0.9},
      "scheduler":{"model":"StepLR","Period":7,"gamma":0.1}
    }
    """

    model_conv = densenet121(weights=Hyperparameter["weight"])

    #model_conv = alexnet(len(self.class_names))
    for param in model_conv.parameters():
        param.requires_grad = True

    # 새로 생성된 모듈의 매개변수는 기본값이 requires_grad=True 임
    #num_ftrs = model_conv.fc.in_features
    #model_conv.fc = nn.Linear(num_ftrs, len(self.class_names))

    model_conv = model_conv.to(self.device)

    if Hyperparameter["Loss Function"]=="CrossEntropyLoss":
      criterion = nn.CrossEntropyLoss()

    # 이전과는 다르게 마지막 계층의 매개변수들만 최적화되는지 관찰
    if Hyperparameter["optimizer"]["model"]=="SGD":
      optimizer_conv = optim.SGD(model_conv.parameters(), lr=Hyperparameter["optimizer"]["lr"], momentum=Hyperparameter["optimizer"]["momentum"])
    elif Hyperparameter["optimizer"]["model"]=="Adam":
       optimizer_conv = optim.Adam(model_conv.parameters(),lr=Hyperparameter["optimizer"]["lr"])

    #optim.SGD(model_conv.fc.parameters(), lr=lr, momentum=momentum)

    # 7 에폭마다 0.1씩 학습률 감소

    if Hyperparameter["scheduler"]["model"]=="StepLR":
      exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=Hyperparameter["scheduler"]["Period"], gamma=Hyperparameter["scheduler"]["gamma"])
    elif Hyperparameter["scheduler"]["model"]=="ReduceLROnPlateau":
      exp_lr_scheduler = lr_scheduler.ReduceLROnPlateau(optimizer_conv)

    return model_conv,criterion,optimizer_conv,exp_lr_scheduler,Hyperparameter
    
  
    
  def RESNET18(self,Hyperparameter):

    #resnet.py 166 line , 195, 279
    
    """
    Hyperparameter = {
      "model":"resnet18",
      "weight":"IMAGENET1K_V1",
      "Loss Function":"CrossEntropyLoss",
      "optimizer":{"model":"Adam","lr":0.001,"momentum":0.9},
      "scheduler":{"model":"StepLR","Period":7,"gamma":0.1},
      "data_feature":{"xtrain_angle":[i for i in range(1,15)],"xtest_angle":[i for i in range(15,25)],"ytrain_angle":[0,30],"ytest_angle":[60]},
      "Domain_Generalization":0, # Domain Generalization (도메인 일반화, 데이터 증진 or mixstyle 등)
    },
    """
    
    model_conv = Resnet18(weights=Hyperparameter["weight"])
    for param in model_conv.parameters():
        param.requires_grad = False

    # 새로 생성된 모듈의 매개변수는 기본값이 requires_grad=True 임
    num_ftrs = model_conv.fc.in_features
    model_conv.fc = nn.Linear(num_ftrs, len(self.class_names))

    model_conv = model_conv.to(self.device)

    if Hyperparameter["Loss Function"]=="CrossEntropyLoss":
      criterion = nn.CrossEntropyLoss()

    # 이전과는 다르게 마지막 계층의 매개변수들만 최적화되는지 관찰
    if Hyperparameter["optimizer"]["model"]=="SGD":
      optimizer_conv = optim.SGD(model_conv.fc.parameters(), lr=Hyperparameter["optimizer"]["lr"], momentum=Hyperparameter["optimizer"]["momentum"])
    elif Hyperparameter["optimizer"]["model"]=="Adam":
       optimizer_conv = optim.Adam(model_conv.fc.parameters(),lr=Hyperparameter["optimizer"]["lr"])

    #optim.SGD(model_conv.fc.parameters(), lr=lr, momentum=momentum)

    # 7 에폭마다 0.1씩 학습률 감소

    if Hyperparameter["scheduler"]["model"]=="StepLR":
      exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=Hyperparameter["scheduler"]["Period"], gamma=Hyperparameter["scheduler"]["gamma"])

    return model_conv,criterion,optimizer_conv,exp_lr_scheduler,Hyperparameter

  def RESNET34(self,Hyperparameter):

    """
    {
      "model":"resnet18",
      "weight":"IMAGENET1K_V1",
      "Loss Function":"CrossEntropyLoss",
      "optimizer":{"model":"SGD","lr":0.001,"momentum":0.9},
      "scheduler":{"model":"StepLR","Period":7,"gamma":0.1}
    }
    """
    #IMAGENET1K_V1
    model_conv = Resnet34(weights=Hyperparameter["weight"])
    for param in model_conv.parameters():
        param.requires_grad = False

    # 새로 생성된 모듈의 매개변수는 기본값이 requires_grad=True 임
    num_ftrs = model_conv.fc.in_features
    model_conv.fc = nn.Linear(num_ftrs, len(self.class_names))

    model_conv = model_conv.to(self.device)

    if Hyperparameter["Loss Function"]=="CrossEntropyLoss":
      criterion = nn.CrossEntropyLoss()

    # 이전과는 다르게 마지막 계층의 매개변수들만 최적화되는지 관찰
    if Hyperparameter["optimizer"]["model"]=="SGD":
      optimizer_conv = optim.SGD(model_conv.fc.parameters(), lr=Hyperparameter["optimizer"]["lr"], momentum=Hyperparameter["optimizer"]["momentum"])
    elif Hyperparameter["optimizer"]["model"]=="Adam":
       optimizer_conv = optim.Adam(model_conv.fc.parameters(),lr=Hyperparameter["optimizer"]["lr"])

    #optim.SGD(model_conv.fc.parameters(), lr=lr, momentum=momentum)

    # 7 에폭마다 0.1씩 학습률 감소

    if Hyperparameter["scheduler"]["model"]=="StepLR":
      exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=Hyperparameter["scheduler"]["Period"], gamma=Hyperparameter["scheduler"]["gamma"])

    return model_conv,criterion,optimizer_conv,exp_lr_scheduler,Hyperparameter

  #아직 설정안됨
  def ALEXNET(self,Hyperparameter):

    """
    {
      "model":"alexnet",
      "weight":"IMAGENET1K_V1",
      "Loss Function":"CrossEntropyLoss",
      "optimizer":{"model":"SGD","lr":0.001,"momentum":0.9},
      "scheduler":{"model":"StepLR","Period":7,"gamma":0.1}
    }
    """

    model_conv = alexnet(weights=Hyperparameter["weight"])

    #model_conv = alexnet(len(self.class_names))
    #for param in model_conv.parameters():
    #    param.requires_grad = True

    # 새로 생성된 모듈의 매개변수는 기본값이 requires_grad=True 임
    #num_ftrs = model_conv.fc.in_features
    #model_conv.fc = nn.Linear(num_ftrs, len(self.class_names))

    model_conv = model_conv.to(self.device)

    if Hyperparameter["Loss Function"]=="CrossEntropyLoss":
      criterion = nn.CrossEntropyLoss()

    # 이전과는 다르게 마지막 계층의 매개변수들만 최적화되는지 관찰
    if Hyperparameter["optimizer"]["model"]=="SGD":
      optimizer_conv = optim.SGD(model_conv.parameters(), lr=Hyperparameter["optimizer"]["lr"], momentum=Hyperparameter["optimizer"]["momentum"])
    elif Hyperparameter["optimizer"]["model"]=="Adam":
       optimizer_conv = optim.Adam(model_conv.parameters(),lr=Hyperparameter["optimizer"]["lr"])

    #optim.SGD(model_conv.fc.parameters(), lr=lr, momentum=momentum)

    # 7 에폭마다 0.1씩 학습률 감소

    if Hyperparameter["scheduler"]["model"]=="StepLR":
      exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=Hyperparameter["scheduler"]["Period"], gamma=Hyperparameter["scheduler"]["gamma"])

    return model_conv,criterion,optimizer_conv,exp_lr_scheduler,Hyperparameter


In [17]:
def START(epoch,batch_size=64,
          Hyperparameter = {
        "model":"resnet18",
        "weight":"IMAGENET1K_V1",
        "Loss Function":"CrossEntropyLoss",
        "optimizer":{"model":"Adam","lr":0.0001,"momentum":0.9},
        "scheduler":{"model":"StepLR","Period":7,"gamma":0.1},
        "data_feature":{"ytrain_angle":[0,30],"ytest_angle":[60]},
        "Domain_Generalization":0, # Domain Generalization (도메인 일반화, 데이터 증진 or mixstyle 등)
        "Data_augmentation":{"image_rotation":0,"ColorJitter":0,"RandomHorizontalFlip":0,"RandomVerticalFlip":0,"RandomGrayscale":0}
        
      },save_file_name="alexnet_sample_data/",inital_path="D:/essay/"):
    
    MPI=market_product_identification(
        inital_path=inital_path,
        using_category=['prod_nm','file_name','yangle','path','rotation'],
        data_feature=Hyperparameter["data_feature"],
        Data_augmentation=Hyperparameter["Data_augmentation"]

    )


    train_v={"label_data":MPI.train,"transform":MPI.train_transform}
    test_v={"label_data":MPI.test,"transform":MPI.test_transform}
    #train_v={"transform":MPI.train_transform}
    #test_v={"transform":MPI.test_transform}

 

    Train,Test = produce_Dataset(train_v,test_v,batch_size,Hyperparameter["data_feature"]["ytest_angle"][0],MPI.a0_path,MPI.a30_path,MPI.a60_path,MPI.nl_idx,Hyperparameter["Data_augmentation"])

    learning_dataset={"train":Train["dataset"],"test":Test["dataset"]}
    learning_dataloader={"train":Train["dataloader"],"test":Test["dataloader"]}
    learning_dataset_size={"train":len(Train["dataset"]),"test":len(Test["dataset"])}

    #new labeling object_type
    class_names=[i for i in range(1,len(MPI.object_type)+1)]

    dl_model=dl_Model(class_names,device)

    model_conv,criterion,optimizer_conv,exp_lr_scheduler,Hyperparameter=dl_model.produce_model(Hyperparameter)



    #torch.backends.cudnn.enabled = False
    Epoch=epoch
    file_n=save_file_name

    #example
    #Hyperparameter={"model":"resnet18","Loss Function":"CrossEntropyLoss","optimizer":{"model":"SGD","lr":0.001,"momentum":0.9},"scheduler":{"model":"StepLR","Period":7,"gamma":0.1}}

    P=inital_path+"result/"+file_n

    #result file produce
    os.mkdir(P)

    Hyperparameter_path=P+"Hyperparameter.txt"

    #Hyperparameter text file produce
    f=open(Hyperparameter_path,"w")
    f.close()
    f=open(Hyperparameter_path,"a")
    f.write(f'model is {Hyperparameter["model"]}.\n')
    f.write(f'weight is {Hyperparameter["weight"]}.\n')
    f.write(f'epoch is {epoch}.\n')
    f.write(f'batch size is {batch_size}.\n')
    f.write(f'ytrain_angle is {Hyperparameter["data_feature"]["ytrain_angle"]}.\n')
    f.write(f'ytest_angle is {Hyperparameter["data_feature"]["ytest_angle"]}.\n')
    f.write(f'Loss Function is {Hyperparameter["Loss Function"]}.\n')
    # "Domain_Generalization":1, # Domain Generalization (도메인 일반화, 데이터 증진 or mixstyle 등)
    #"Data_augmentation":{"image_rotation":0,"ColorJitter":0,"RandomHorizontalFlip":0,"RandomVerticalFlip":0,"RandomGrayscale":0}
    f.write(f'Domain_Generalization is {Hyperparameter["Domain_Generalization"]}.\n')
    f.write(f'Data_augmentation\n')
    f.write(f'image_rotation is {Hyperparameter["Data_augmentation"]["image_rotation"]}.\n')
    f.write(f'ColorJitter is {Hyperparameter["Data_augmentation"]["ColorJitter"]}.\n')
    f.write(f'RandomHorizontalFlip is {Hyperparameter["Data_augmentation"]["RandomHorizontalFlip"]}.\n')
    f.write(f'RandomGrayscale is {Hyperparameter["Data_augmentation"]["RandomGrayscale"]}.\n')

    f.write(f'optimizer is {Hyperparameter["optimizer"]["model"]}. lr is {Hyperparameter["optimizer"]["lr"]}, momentum is {Hyperparameter["optimizer"]["momentum"]}.\n')
    f.write(f'scheduler is {Hyperparameter["scheduler"]["model"]}. Period is {Hyperparameter["scheduler"]["Period"]}, gamma is {Hyperparameter["scheduler"]["gamma"]}.\n')

    f.close()

    # loss,acc
    TRAIN=[]
    TEST=[]

    since = time.time()
    best_acc = 0.0
    best_loss=0.0
    idx=0
    #print(learning_dataset_size["train"])
    #print(learning_dataset_size["val"])
    #print(learning_dataset_size["test"])

    train_size=[]
    test_size=[]
    #print("##@@##")
    #print(MPI.object_stack)
    for i in range(len(MPI.object_stack)):
        QE=Hyperparameter['data_feature']['ytrain_angle']#MPI.object_stack[i][Hyperparameter['data_feature']['ytrain_angle']]
        #print(QE)
        train_size.append(MPI.object_stack[i][str(QE[0])]+MPI.object_stack[i][str(QE[1])])
        test_size.append(MPI.object_stack[i][str(Hyperparameter['data_feature']['ytest_angle'][0])])

    #print(train_size)
    #print(test_size)


    for epoch in range(Epoch):
        print(f'Epoch {epoch}/{Epoch - 1}')
        print('-' * 10)

        #print("train")
        #print("="*20)

        #return epoch_loss,epoch_acc,model,optimizer,scheduler
        #print("XCVXCV")
        epoch_loss,epoch_acc,model_conv,optimizer_conv,exp_lr_scheduler=train_model(model_conv, 
                                                                                    criterion, 
                                                                                    optimizer_conv,
                                                                                    exp_lr_scheduler, 
                                                                                    learning_dataloader["train"],
                                                                                    learning_dataset_size["train"],
                                                                                    batch_size,train_size,Hyperparameter["Data_augmentation"])
        TRAIN.append([epoch_loss,epoch_acc])

        #print("val")
        #print("="*20)

        #return epoch_loss,epoch_acc,model,optimizer,scheduler

        # 모델을 깊은 복사(deep copy)함
        if epoch_acc > best_acc:
            best_acc = epoch_acc
            idx=epoch
            best_loss=epoch_loss

        torch.save(model_conv.state_dict(), P+'/checkpoint_epoch_'+str(epoch+1)+'.pth')


    time_elapsed = time.time() - since

    print()
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best train Acc: {best_acc:4f}')

    result_val_text_path=P+"/result_train.txt"

    #val result save
    f=open(result_val_text_path,"w")
    f.close()
    f=open(result_val_text_path,"a")
    for i in range(Epoch):
        f.write(f"epoch_{i+1}\n")
        f.write(f"epoch_{i+1} train loss : {TRAIN[i][0]}, train acc : {TRAIN[i][1]}\n")

    f.write("\n")
    f.write(f"best val acc's epoch is epoch_{idx+1}.\n")
    f.write(f"best val acc is {best_acc} and this acc's loss is {best_loss}.\n")
    f.close()

    ############################################

    since = time.time()
    best_acc = 0.0
    best_loss=0.0
    idx=0

    for epoch in range(Epoch):
        print(f'Epoch {epoch}/{Epoch - 1}')
        print('-' * 10)

        #print("test")
        #print("="*20)

        model_conv.load_state_dict(torch.load(P+'/checkpoint_epoch_'+str(epoch+1)+'.pth'))
        epoch_loss,epoch_acc,model_conv=test_model(model_conv, criterion, learning_dataloader["test"],learning_dataset_size["test"],test_size,Hyperparameter["Data_augmentation"])
        # 모델을 깊은 복사(deep copy)함
        if epoch_acc > best_acc:
            best_acc = epoch_acc
            idx=epoch
            best_loss=epoch_loss

        #torch.save(model_conv.state_dict(), P+'/checkpoint_epoch_'+str(epoch+1)+'.pth')
        TEST.append([epoch_loss,epoch_acc])

    time_elapsed = time.time() - since
    #print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best test Acc: {best_acc:4f}')


    f=open(P+"/result_test.txt","w")
    f.close()
    f=open(P+"/result_test.txt","a")
    for i in range(Epoch):
      f.write(f"epoch_{i+1}\n")
      f.write(f"epoch_{i+1} test loss : {TEST[i][0]}, test acc : {TEST[i][1]}\n")

    f.write("\n")
    f.write(f"best test acc's epoch is epoch_{idx+1}.\n")
    f.write(f"best test acc is {best_acc} and this acc's loss is {best_loss}.\n")
    f.close()


    

In [18]:
"""
기본 설정
F라는 폴더안에 파일들이 있을때 inital_path 에 F 폴더 위치를 설정해줘야함
모든 path 는 뒤에 \를 붙여야함
F 폴더 안에는 dataset 이라는 데이터 파일, result 폴더가 있어야 함


1분 16초에 5epoch rotation x 가 돌기 때문에 epoch 해도 얼마 안걸릴듯

vsc 에선 desenet 터짐
resnet 은 batch_size 64 도 돌지만 나머지는 돌지 않기 때문에 colab 에서 16 이하로 돌려야됨
gpu 메모리 초과가 뜨면 batch_size 를 줄여라
batch_size 줄이고 lr 도 줄이고
https://inhovation97.tistory.com/32
https://blog.joonas.io/193
"""

angle_stack=[0,30,60]
for i in range(3):
  ytrain_angle=[]
  ytest_angle=[angle_stack[i]]
  for g in range(3):
    if i!=g:
      ytrain_angle.append(angle_stack[g])

  #print(ytrain_angle)
  #print(ytest_angle)
  #print(f"ytrain_angle : {ytrain_angle}.")
  #print(f"ytest_angle : {ytest_angle}.")


  START(
      epoch = 1,batch_size=16,
      Hyperparameter = {
        "model":"resnet18",
        "weight":"IMAGENET1K_V1",
        "Loss Function":"CrossEntropyLoss",
        "optimizer":{"model":"Adam","lr":0.0001,"momentum":0.9},
        "scheduler":{"model":"StepLR","Period":7,"gamma":0.1},
        "data_feature":{"ytrain_angle":ytrain_angle,"ytest_angle":ytest_angle},
        "Domain_Generalization":1, # Domain Generalization (도메인 일반화, 데이터 증진 or mixstyle 등)
        "Data_augmentation":{"image_rotation":1,"ColorJitter":1,"RandomHorizontalFlip":1,"RandomVerticalFlip":1,"RandomGrayscale":1}
        
      },
      save_file_name = "test1_"+str(angle_stack[i])+"/",
      inital_path = "D:/essay/thesis/"#"/content/drive/MyDrive/01_고원규/"
      
      )
 



Epoch 0/0
----------
train Loss: 1.7573 Acc: 0.3526

Training complete in 0m 25s
Best train Acc: 0.352593
Epoch 0/0
----------
test Loss: 1.5363 Acc: 0.5200
Best test Acc: 0.520045
Epoch 0/0
----------
train Loss: 1.8393 Acc: 0.2873

Training complete in 0m 23s
Best train Acc: 0.287262
Epoch 0/0
----------
test Loss: 1.5625 Acc: 0.4600
Best test Acc: 0.459952
Epoch 0/0
----------
train Loss: 1.8170 Acc: 0.3058

Training complete in 0m 23s
Best train Acc: 0.305846
Epoch 0/0
----------
test Loss: 1.6874 Acc: 0.4708
Best test Acc: 0.470766
